# SSCN and PointNet Layer Implementation Assignment

SSCN uses the preprocessing method used in [Volumetric and Multi-View CNNs for Object Classification on 3D Data](https://arxiv.org/abs/1604.03265)

We are going to use a simpler method that doesn't require another CNN, where we voxelize the given .off files and project them from multiple angles to get a 2d image
we can use.

### Download binvox and dataset

In [2]:
!test -f binvox || wget https://www.patrickmin.com/binvox/linux64/binvox

In [3]:
!chmod +x binvox

In [5]:
!./download_dataset.sh > /dev/null

### Runs script to voxelize `.off` file
`/dev/null` pipe is to stop it from breaking stdout, if the program is not working, remove it to debug

In [4]:
!./voxelize.sh ModelNet10/bed/train/bed_0010.off > /dev/null

In [1]:
!g++ -std=c++11 read_vox.cpp -o read_vox

In [3]:
!./read_vox ModelNet10/bed/train/bed_0010.binvox

reading binvox version 1
  read 143417 voxels
Writing voxel data to ASCII file...
done



## Parse voxel file into C++ memory

In [10]:
#include<fstream>
#include<iostream>
#include<vector>
#include<string>
using namespace std;



In [17]:
typedef vector<vector<vector<char>>> v3d;
typedef vector<vector<char>> v2d;


Read all voxel data from voxels.txt and output in 3d vector

In [36]:
v3d read_vox(){
    ifstream voxels("voxels.txt");
    string t;
    int x, y, z;
    // ignore irrelevant 
    voxels >> t >> t >> t >> t;
    voxels >> x >> y >> z;
    voxels >> t >> t >> t >> t;
    voxels >> t >> t >> t;
    vector<vector<vector<char>>> data(x, vector<vector<char>>(z, vector<char>(y, 0)));
    // Read data into vec
    // format is [x][z][y]
    for(int i = 0; i < x; i++){
        for(int j = 0; j < z; j++){
            for(int k = 0; k < y; k++){
                char v;
                voxels >> v;
                data[i][j][k] = v;
            }
        }
        
    }
    return data;
}


In [37]:
char max(char x, char y) {
    return x > y ? x : y;
}

In [38]:
// Project onto x y or z
// sweeps across given axis
// 0 = x, 1 = z, 2 = y
v2d project(const v3d &data, int axis, int x_size, int y_size){
    v2d out(x_size, vector<char>(y_size, 0));
    if(axis == 0){
        for(int i = 0; i < data.size(); i++){
            for(int j = 0; j < data[0].size(); j++){
                for(int k = 0; k < data[0][0].size(); k++){
                    out[j][k] = max(out[j][k], data[i][j][k]);
                }
            }
        }
    } else if(axis == 1){
        for(int i = 0; i < data.size(); i++){
            for(int j = 0; j < data[0].size(); j++){
                for(int k = 0; k < data[0][0].size(); k++){
                    out[j][k] = max(out[j][k], data[j][i][k]);
                }
            }
        }
    } else{
        for(int i = 0; i < data.size(); i++){
            for(int j = 0; j < data[0].size(); j++){
                for(int k = 0; k < data[0][0].size(); k++){
                    out[j][k] = max(out[j][k], data[j][k][i]);
                }
            }
        }
    }
    return out;
}

In [39]:
void print_2d(const v2d &data){
    for(auto i : data){
        for (auto j : i){
            cout << j << " ";
        }
        cout << endl;
    }
}

In [43]:
// Get our 3d vector of intputs
v3d data = read_vox();
int x = data.size();
int z = data[0].size();
int y = data[0][0].size();
cout << "x: " << x << endl;
cout << "y: " << y << endl;
cout << "z: " << z << endl;
v2d p = project(data, 2, x, y);
print_2d(p);

x: 256
y: 256
z: 256
0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 